####Welcome to the example notebook for FIFEforSpark! 

You may recognize the following example from FIFE's notebook found [here](https://github.com/IDA-HumanCapital/fife/blob/master/examples/country_leadership.ipynb). Like that example notebook, we use the July 2020 edition of the Rulers, Elections, and Irregular Governance dataset (REIGN) dataset, a monthly panel of national leaders and political conditions since January 1950. We load the REIGN data directly from its online archive.

First, we import the necessary packages. In this case, we import SparkFiles which is required to read the data in from the url, in addition to several fifeforspark modules.

In [0]:
import pyspark
from pyspark import SparkFiles
import fifeforspark
from fifeforspark.utils import create_example_data2
from fifeforspark.processors import PanelDataProcessor
from fifeforspark.lgb_modelers import LGBSurvivalModeler

Now that we have the necessary packages loaded, we read in the data from a url:

In [0]:
url = "https://www.dl.dropboxusercontent.com/s/3tdswu2jfgwp4xw/REIGN_2020_7.csv?dl=0"
spark.sparkContext.addFile(url)

df = spark.read.csv("file://"+SparkFiles.get("REIGN_2020_7.csv"), header=True, inferSchema= True)

The data is stored in a Spark DataFrame which is different than you may expect if you are more familiar with FIFE. Let's examine our data a bit more.

In [0]:
df.show(10)

+--------------------+-----------+-----------+------+-----+-------+----+----+--------------+-------------+--------------------+------------+-------+-------+--------+--------------+------------+---------------+----------+-----------+-----------+----------+-------------+---------------+--------------+-------------+-------------+---------------+-------+------------+---------+---------+-------------+------+----------+-------------------+------------+-----------+
 country-leader| year-month| country| year|month|elected| age|male|militarycareer|tenure_months| government|anticipation|ref_ant|leg_ant|exec_ant|irreg_lead_ant|election_now|election_recent|leg_recent|exec_recent|lead_recent|ref_recent|direct_recent|indirect_recent|victory_recent|defeat_recent|change_recent|nochange_recent|delayed|lastelection| loss|irregular|prev_conflict|pt_suc|pt_attempt| precip| couprisk|pctile_risk|
+--------------------+-----------+-----------+------+-----+-------+----+----+--------------+-------------+--------------------+------------+-------+-------+--------+--------------+------------+---------------+----------+-----------+-----------+----------+-------------+---------------+--------------+-------------+-------------+---------------+-------+------------+---------+---------+-------------+------+----------+-------------------+------------+-----------+
Afghanistan:Abdul...|1.9710901E7|Afghanistan|1971.0| 9.0| 0.0|61.0| 1| 0.0| 4.0| Monarchy| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 6.222576| 6.222576| 6.222576| 0.0| 0.0| 0.0| -1.84107666146101| null| null|
Afghanistan:Abdul...|1.9720101E7|Afghanistan|1972.0| 1.0| 0.0|62.0| 1| 0.0| 8.0| Monarchy| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 6.2304816|6.2304816|6.2304816| 0.0| 0.0| 0.0| -0.637760953387462| null| null|
 Afghanistan:Amin|1.9790801E7|Afghanistan|1979.0| 8.0| 0.0|50.0| 1| 0.0| 6.0| Party-Personal| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 2.8332133|2.8332133|2.8332133| 1.0| 0.0| 0.0| 0.214798026241846| 0.012939446| 1.0159242|
 Afghanistan:Amin|1.9791001E7|Afghanistan|1979.0| 10.0| 0.0|50.0| 1| 0.0| 8.0| Party-Personal| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 2.944439| 2.944439| 2.944439| 1.0| 0.0| 0.0| 0.245955774125835| 0.018041687| 1.0320948|
Afghanistan:Ashra...|2.0151101E7|Afghanistan|2015.0| 11.0| 1.0|66.0| 1| 0.0| 15.0|Presidential Demo...| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 2.8903718|2.8903718| 4.89784| 2.0| 0.0| 0.0|0.00139763311041197| 0.00361316| 0.8759844|
Afghanistan:Ashra...|2.0160301E7|Afghanistan|2016.0| 3.0| 1.0|67.0| 1| 0.0| 19.0|Presidential Demo...| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 3.0910425|3.0910425|4.9272537| 2.0| 0.0| 0.0| -0.317054186285647|0.0010842119| 0.6260331|
Afghanistan:Ashra...|2.0160901E7|Afghanistan|2016.0| 9.0| 1.0|67.0| 1| 0.0| 25.0|Presidential Demo...| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 3.3322046|3.3322046|4.9698133| 2.0| 0.0| 0.0| 0.0730770508737204|0.0010030536| 0.5974805|
Afghanistan:Ashra...|2.0170201E7|Afghanistan|2017.0| 2.0| 1.0|68.0| 1| 0.0| 30.0|Presidential Demo...| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 3.4965076|3.4965076|5.0039463| 2.0| 0.0| 0.0| 0.460152116601137|9.1615773E-4| 0.5673159|
Afghanistan:Ashra...|2.0190801E7|Afghanistan|2019.0| 8.0| 1.0|70.0| 1| 0.0| 60.0|Presidential Demo...| 1.0| 0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0| 4.1431346|4.1431346| 5.187386| 2.0| 0.0| 0.0| 1.47129165038672| 9.935072E-4| 0.5939999|
Afghanistan:Burha...|1.9930301E7|Afghanistan|1993.0| 3.0| 0.0|54.0| 1| 0.0| 10.0| Warlordism| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.

This isn't very pleasant to look at; however, the advantage of using a Spark DataFrame here (even though this could fit on one node) is that it's distributed. Fortunately, we can use the display() function to output a cleaner dataframe.

In [0]:
display(df)

country-leader,year-month,country,year,month,elected,age,male,militarycareer,tenure_months,government,anticipation,ref_ant,leg_ant,exec_ant,irreg_lead_ant,election_now,election_recent,leg_recent,exec_recent,lead_recent,ref_recent,direct_recent,indirect_recent,victory_recent,defeat_recent,change_recent,nochange_recent,delayed,lastelection,loss,irregular,prev_conflict,pt_suc,pt_attempt,precip,couprisk,pctile_risk
Afghanistan:Abdul Zahir,1.9710901E7,Afghanistan,1971.0,9.0,0.0,61.0,1,0.0,4.0,Monarchy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.222576,6.222576,6.222576,0.0,0.0,0.0,-1.84107666146101,null,null
Afghanistan:Abdul Zahir,1.9720101E7,Afghanistan,1972.0,1.0,0.0,62.0,1,0.0,8.0,Monarchy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.2304816,6.2304816,6.2304816,0.0,0.0,0.0,-0.637760953387462,null,null
Afghanistan:Amin,1.9790801E7,Afghanistan,1979.0,8.0,0.0,50.0,1,0.0,6.0,Party-Personal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.8332133,2.8332133,2.8332133,1.0,0.0,0.0,0.214798026241846,0.012939446,1.0159242
Afghanistan:Amin,1.9791001E7,Afghanistan,1979.0,10.0,0.0,50.0,1,0.0,8.0,Party-Personal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.944439,2.944439,2.944439,1.0,0.0,0.0,0.245955774125835,0.018041687,1.0320948
Afghanistan:Ashraf Ghani,2.0151101E7,Afghanistan,2015.0,11.0,1.0,66.0,1,0.0,15.0,Presidential Democracy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.8903718,2.8903718,4.89784,2.0,0.0,0.0,0.00139763311041197,0.00361316,0.8759844
Afghanistan:Ashraf Ghani,2.0160301E7,Afghanistan,2016.0,3.0,1.0,67.0,1,0.0,19.0,Presidential Democracy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0910425,3.0910425,4.9272537,2.0,0.0,0.0,-0.317054186285647,0.0010842119,0.6260331
Afghanistan:Ashraf Ghani,2.0160901E7,Afghanistan,2016.0,9.0,1.0,67.0,1,0.0,25.0,Presidential Democracy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.3322046,3.3322046,4.9698133,2.0,0.0,0.0,0.0730770508737204,0.0010030536,0.5974805
Afghanistan:Ashraf Ghani,2.0170201E7,Afghanistan,2017.0,2.0,1.0,68.0,1,0.0,30.0,Presidential Democracy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.4965076,3.4965076,5.0039463,2.0,0.0,0.0,0.460152116601137,9.1615773E-4,0.5673159
Afghanistan:Ashraf Ghani,2.0190801E7,Afghanistan,2019.0,8.0,1.0,70.0,1,0.0,60.0,Presidential Democracy,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.1431346,4.1431346,5.187386,2.0,0.0,0.0,1.47129165038672,9.935072E-4,0.5939999
Afghanistan:Burhanuddin Rabbani,1.9930301E7,Afghanistan,1993.0,3.0,0.0,54.0,1,0.0,10.0,Warlordism,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0943446,2.4849067,4.0943446,1.0,0.0,0.0,0.874634005217249,0.016689193,1.0287888


Much better! Let's see how many partitions we have.

In [0]:
df.rdd.getNumPartitions()

Out[17]: 8

Wow! The data is split across 8 partitions! We can always change this number, but for this example we will leave it as 8.

Next, we make some changes to the data to prepare it for analysis

In [0]:
from pyspark.sql.functions import lit, lpad, col, concat, date_format
from pyspark.sql.types import DateType


df = df.withColumn('country-leader', concat(col('country'),lit(":"),col('leader')))
df = df.withColumn('year-month', concat(col('year').cast('integer').cast('string'),lit("-"), lpad(col('month').cast('integer').cast('string'), 2, "0"), lit("-"),lit("01")))

df = df.withColumn('year-month', df['year-month'].cast(DateType()))

cols = ['country-leader', 'year-month'] + [x for x in df.columns if x not in ["ccode", "country-leader", "leader", "year-month"]]
df = df.select(cols)
total_obs = df.count()
df = df.drop_duplicates(subset = ["country-leader", "year-month"])
n_duplicates = total_obs - df.count()
print(f"{n_duplicates} observations with a duplicated identifier pair deleted.")

df = df.withColumn('year-month',10000*date_format(df['year-month'], "y") +
                                100*date_format(df['year-month'], "M") +
                                date_format(df['year-month'], "d"))

7 observations with a duplicated identifier pair deleted.

Now that we have created unique identifiers for the individual and time, we pass the data through the Panel Data Processor, specifying a value of 4 for 'TEST_INTERVALS' as we want to test the last 4 periods. For the time being, we transform the time_id back to a numeric feature given constraints regarding datetime functionality.

In [0]:
test_intervals = 4
processor = PanelDataProcessor(data=df, config = {'TEST_INTERVALS': test_intervals}, shuffle_parts = 20)
processor.build_processed_data()
display(processor.data)

Time identifier column name not given; assumed to be second-leftmost column (year-month)
Individual identifier column name not given; assumed to be leftmost column (country-leader)

country-leader,year-month,country,year,month,elected,age,male,militarycareer,tenure_months,government,anticipation,ref_ant,leg_ant,exec_ant,irreg_lead_ant,election_now,election_recent,leg_recent,exec_recent,lead_recent,ref_recent,direct_recent,indirect_recent,victory_recent,defeat_recent,change_recent,nochange_recent,delayed,lastelection,loss,irregular,prev_conflict,pt_suc,pt_attempt,precip,couprisk,pctile_risk,_period,_predict_obs,_test,_validation,_maximum_lead,_spell,_duration,_event_observed
Afghanistan:Abdallah Yakta,1.9671001E7,Afghanistan,1967.0,10.0,0.0,53.0,1,0.0,1.0,Monarchy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.1246834,6.1246834,6.1246834,0.0,0.0,0.0,0.0187039816454769,null,null,213,false,false,true,629,0,1,true
Afghanistan:Abdallah Yakta,1.9671101E7,Afghanistan,1967.0,11.0,0.0,53.0,1,0.0,2.0,Monarchy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.126869,6.126869,6.126869,0.0,0.0,0.0,0.17923993006129,null,null,214,false,false,true,628,0,0,true
Afghanistan:Abdul Zahir,1.9710601E7,Afghanistan,1971.0,6.0,0.0,61.0,1,0.0,1.0,Monarchy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.216606,6.216606,6.216606,0.0,0.0,0.0,-1.80655395371697,null,null,257,false,false,false,585,0,18,true
Afghanistan:Abdul Zahir,1.9710701E7,Afghanistan,1971.0,7.0,0.0,61.0,1,0.0,2.0,Monarchy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.2186003,6.2186003,6.2186003,0.0,0.0,0.0,-1.79781203785734,null,null,258,false,false,false,584,0,17,true
Afghanistan:Abdul Zahir,1.9710801E7,Afghanistan,1971.0,8.0,0.0,61.0,1,0.0,3.0,Monarchy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.22059,6.22059,6.22059,0.0,0.0,0.0,-1.81513438987383,null,null,259,false,false,false,583,0,16,true
Afghanistan:Abdul Zahir,1.9710901E7,Afghanistan,1971.0,9.0,0.0,61.0,1,0.0,4.0,Monarchy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.222576,6.222576,6.222576,0.0,0.0,0.0,-1.84107666146101,null,null,260,false,false,false,582,0,15,true
Afghanistan:Abdul Zahir,1.9711001E7,Afghanistan,1971.0,10.0,0.0,61.0,1,0.0,5.0,Monarchy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.2245584,6.2245584,6.2245584,0.0,0.0,0.0,-1.90343827157157,null,null,261,false,false,false,581,0,14,true
Afghanistan:Abdul Zahir,1.9711101E7,Afghanistan,1971.0,11.0,0.0,61.0,1,0.0,6.0,Monarchy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.2265368,6.2265368,6.2265368,0.0,0.0,0.0,-1.7699616319019,null,null,262,false,false,false,580,0,13,true
Afghanistan:Abdul Zahir,1.9711201E7,Afghanistan,1971.0,12.0,0.0,61.0,1,0.0,7.0,Monarchy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.228511,6.228511,6.228511,0.0,0.0,0.0,-1.77719344534756,null,null,263,false,false,false,579,0,12,true
Afghanistan:Abdul Zahir,1.9720101E7,Afghanistan,1972.0,1.0,0.0,62.0,1,0.0,8.0,Monarchy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.2304816,6.2304816,6.2304816,0.0,0.0,0.0,-0.637760953387462,null,null,264,false,false,false,578,0,11,true


Now, we build the model. You can pass parameters into the model that will be passed to lightgbm as well.

In [0]:
modeler = LGBSurvivalModeler(data=processor.data)
modeler.build_model(n_intervals=test_intervals)

Now we want to see how well our model performs on the test data and take a look at the predictions

In [0]:
metrics = modeler.evaluate()

Now evaluating lead length: 1 of 4
Now evaluating lead length: 2 of 4
Now evaluating lead length: 3 of 4
Now evaluating lead length: 4 of 4

In [0]:
metrics

Out[23]:

,AUROC,Predicted Share,Actual Share,True Positives,False Negatives,False Positives,True Negatives,Other Metrics:
Lead Length,,,,,,,,
1,0.884536,0.988373,0.974874,194,0,4,1,
2,0.914435,0.972729,0.964824,191,1,5,2,
3,0.867836,0.955600,0.954774,188,2,7,2,
4,0.841191,0.934313,0.934673,184,2,10,3,


And finally, we want to forecast future survival probabilities for country-leaders in the last period

In [0]:
forecasts = modeler.forecast()
forecasts.head(20)

Out[24]:

,1-period Survival Probability,2-period Survival Probability,3-period Survival Probability,4-period Survival Probability
0,0.997633,0.997279,0.995974,0.991749
1,0.985465,0.971924,0.951843,0.939733
2,0.995495,0.988515,0.983817,0.979070
3,0.988525,0.964400,0.954410,0.951114
4,0.993996,0.992956,0.991646,0.986531
5,0.991682,0.990638,0.983942,0.978513
6,0.993015,0.986875,0.984801,0.981708
7,0.994999,0.994422,0.981580,0.972685
8,0.996849,0.993643,0.989943,0.985863
9,0.989168,0.916900,0.817928,0.776553
